<a href="https://colab.research.google.com/github/Debopam-Chowdhury/VFL_Sem_8_project/blob/Vivek/trial_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install Dependencies
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install transformers peft
!pip install opacus
!pip install pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 23.6 MB/s eta 0:00:00


In [2]:
#Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

from torch_geometric.nn import GCNConv, GATConv
from transformers import AutoModel, AutoTokenizer
from peft import LoraConfig, get_peft_model
from opacus import PrivacyEngine


In [ ]:
#Load Dataset
DATA_DIR = "/content/dataset"

movies_df = pd.read_csv(os.path.join(DATA_DIR, "movies.csv"))
credits_df = pd.read_csv(os.path.join(DATA_DIR, "credits.csv"))
genome_df = pd.read_csv(os.path.join(DATA_DIR, "genome.csv"))
ratings_df = pd.read_csv(os.path.join(DATA_DIR, "ratings.csv"))


In [ ]:
# merge everything
df = movies_df.merge(credits_df, on="id")
df = df.merge(genome_df, on="id")
df = df.merge(ratings_df, left_on="id", right_on="movieId")


In [ ]:
#Dataset Class (Real Data)
class VFLMovieDataset(Dataset):
    def __init__(self, dataframe, data_dir):
        self.df = dataframe.reset_index(drop=True)
        self.data_dir = data_dir
        self.tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

        self.image_transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # ----- Image -----
        img_path = os.path.join(self.data_dir, "posters", f"{row['id']}.jpg")
        image = Image.open(img_path).convert("RGB")
        image = self.image_transform(image)

        # ----- Overview text -----
        text_tokens = self.tokenizer(
            str(row["overview"]),
            padding="max_length",
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )

        # ----- Credits text -----
        credit_tokens = self.tokenizer(
            str(row["cast"]) + " " + str(row["crew"]),
            padding="max_length",
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )

        # ----- Genome numeric -----
        genome_features = torch.tensor(
            row.filter(regex="genome_").values,
            dtype=torch.float
        )

        rating = torch.tensor([row["rating"]], dtype=torch.float)

        return {
            "image": image,
            "text": {k: v.squeeze(0) for k, v in text_tokens.items()},
            "credits": {k: v.squeeze(0) for k, v in credit_tokens.items()},
            "genome": genome_features,
            "rating": rating
        }


In [ ]:
#Define 4 Clients


In [ ]:
#Client 1 — Image + GNN
class ClientImage(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        for param in self.cnn.parameters():
            param.requires_grad = False  # freeze vision

        self.cnn.fc = nn.Linear(512, embed_dim)
        self.gnn = GCNConv(embed_dim, embed_dim)

    def forward(self, image, edge_index):
        x = self.cnn(image)
        x = self.gnn(x, edge_index)
        return x


In [ ]:
#Client 2 — Overview Text (LoRA + GNN)
class ClientText(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()

        base_model = AutoModel.from_pretrained("xlm-roberta-base")

        lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            target_modules=["query", "value"],
            lora_dropout=0.05
        )

        self.encoder = get_peft_model(base_model, lora_config)
        self.proj = nn.Linear(768, embed_dim)
        self.gnn = GCNConv(embed_dim, embed_dim)

    def forward(self, tokens, edge_index):
        x = self.encoder(**tokens).last_hidden_state[:,0,:]
        x = self.proj(x)
        x = self.gnn(x, edge_index)
        return x


In [ ]:
#Client 3 — Credits Text (Frozen + Head FT)
class ClientCredits(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.encoder = AutoModel.from_pretrained("xlm-roberta-base")
        for p in self.encoder.parameters():
            p.requires_grad = False

        self.proj = nn.Linear(768, embed_dim)
        self.gnn = GCNConv(embed_dim, embed_dim)

    def forward(self, tokens, edge_index):
        x = self.encoder(**tokens).last_hidden_state[:,0,:]
        x = self.proj(x)
        x = self.gnn(x, edge_index)
        return x


In [ ]:
#Client 4 — Genome Numeric
class ClientGenome(nn.Module):
    def __init__(self, input_dim, embed_dim=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim,128),
            nn.ReLU(),
            nn.Linear(128,embed_dim)
        )
        self.gnn = GCNConv(embed_dim, embed_dim)

    def forward(self, x, edge_index):
        x = self.mlp(x)
        x = self.gnn(x, edge_index)
        return x


In [ ]:
#Server Fusion (GAT + DP)
class FusionServer(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.gat = GATConv(input_dim, 256, heads=2)
        self.out = nn.Linear(512,1)

    def forward(self, x, edge_index):
        x = self.gat(x, edge_index)
        return self.out(x)


In [ ]:
#Training Loop (VFL)
def train_epoch(clients, server, loader, optimizer, device):
    server.train()
    total_loss = 0
    criterion = nn.MSELoss()

    for batch in loader:
        image = batch["image"].to(device)
        text = {k:v.to(device) for k,v in batch["text"].items()}
        credits = {k:v.to(device) for k,v in batch["credits"].items()}
        genome = batch["genome"].to(device)
        rating = batch["rating"].to(device)

        # simple fully-connected graph
        num_nodes = image.size(0)
        edge_index = torch.combinations(torch.arange(num_nodes), r=2).t().to(device)

        e1 = clients[0](image, edge_index)
        e2 = clients[1](text, edge_index)
        e3 = clients[2](credits, edge_index)
        e4 = clients[3](genome, edge_index)

        fused = torch.cat([e1,e2,e3,e4], dim=-1)

        output = server(fused, edge_index)

        loss = criterion(output, rating)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss/len(loader)


In [ ]:
#Initialize Everything
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = VFLMovieDataset(df, DATA_DIR)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

client1 = ClientImage().to(device)
client2 = ClientText().to(device)
client3 = ClientCredits().to(device)
client4 = ClientGenome(input_dim=genome_df.filter(regex="genome_").shape[1]).to(device)

clients = [client1, client2, client3, client4]

server = FusionServer(input_dim=128+128+128+64).to(device)
optimizer = optim.Adam(server.parameters(), lr=1e-3)


In [ ]:
#Apply Differential Privacy
privacy_engine = PrivacyEngine()

server, optimizer, loader = privacy_engine.make_private(
    module=server,
    optimizer=optimizer,
    data_loader=loader,
    noise_multiplier=1.0,
    max_grad_norm=0.5
)


In [ ]:
#Train
for epoch in range(10):
    loss = train_epoch(clients, server, loader, optimizer, device)
    eps = privacy_engine.get_epsilon(delta=1e-5)
    print(f"Epoch {epoch+1} | Loss: {loss:.4f} | ε={eps:.2f}")
